## Spatial interpolation and approximation I: methods
Resources:

* [
GRASS GIS overview and manual](https://grass.osgeo.org/grass76/manuals/index.html)
*  [Recommendations](data_acquisition.html#commands)
and [tutorial](./grass_intro.html)
how to use GUI from the first assignment


### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
# This is a quick introduction into Jupyter Notebook.
# Python code can be executed like this:
a = 6
b = 7
c = a * b
print("Answer is", c)
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In startup pannel set GRASS GIS Database Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For GRASS Location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for GRASS Mapset create a new mapset (called e.g. HW_interpolation_1).
Click Start GRASS session.


Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

### Compute Voronoi polygons
Display the polygons with centroids.
Find the column name where z is stored and convert the polygons to raster.
Compute aspect to evaluate the surface geometry.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('d.erase')
gs.run_command('v.voronoi', input="elev_lid792_randpts", output="elev_vor")
gs.run_command('d.vect', map="elev_vor", size="1", type="area,centroid", flags='c')
print(gs.read_command('v.info', map="elev_vor", flags='c'))
gs.run_command('v.to.rast', input="elev_vor", output="elev_vor_1m", attrcolumn="value", use="attr")
gs.run_command('r.colors', map="elev_vor_1m", color="elevation")
gs.run_command('r.slope.aspect', elevation="elev_vor_1m", aspect="asp_vor_1m")
Image(filename="map.png")

Display the resulting elevation map and aspect map.

In [ ]:
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_vor_1m")
gs.run_command('d.rast', map="asp_vor_1m")
gs.run_command('d.vect', map="elev_lid792_randpts", size="1", color="red")
Image(filename="map.png")

Optionally, you can view it in 3D perspective
(switch off all layers except for elev_vor_1m and switch to 3D view).

### Interpolation using IDW
Set region and resolution, find a column name where z is stored.
Interpolate DEM using IDW, check the result using aspect.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
print(gs.read_command('v.info', map="elev_lid792_randpts", flags='c'))
gs.run_command('v.surf.idw', input="elev_lid792_randpts", output="elev_idw_1m", column="value")
gs.run_command('r.colors', map="elev_idw_1m", color="elevation")
gs.run_command('r.slope.aspect', elevation="elev_idw_1m", aspect="asp_idw_1m")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_idw_1m")
gs.run_command('d.rast', map="asp_idw_1m")
gs.run_command('d.vect', map="elev_lid792_randpts", size="2", color="red")
Image(filename="map.png")

Design experiment that elucidates the impact of IDW parameters on the surface,
focus on the impact of:

* exponent e.g., power=0.5, 1, 5 (2 is the default)
* number of neighboring points e.g., npoint=1, 5, 20, 60 (12 is the default)

Include selected images (e.g. hillshade or aspect)
and _relevant_ stats (e.g., mean, min, max from _r.univar_, histogram)
that highlight the differences in the resulting surfaces into your report.

Check the surface interpolated with default parameters using 3D view.
Do not forget to switch off everything except for the interpolated elevations and
set fine resolution to 1.
You can use constant color for the surface to highlight its structure.
Save an image for your report.

### Compute DEM from contours
Compute DEM from contours using linear interpolation between isolines:

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('v.to.rast', input="elev_lid792_cont1m", output="el_lid792_cont1m", attrcolumn="level", use="attr")
gs.run_command('r.surf.contour', input="el_lid792_cont1m", output="el_rcont")
gs.run_command('r.colors', map="el_rcont", color="elevation")

Check the result using a 2D aspect map or view el_rcont in 3D.
In 3D set view from SE and light from NW to reveal subtle geometry.

In [ ]:
gs.run_command('r.slope.aspect', elevation="el_rcont", aspect="asp_rcont")
gs.run_command('d.rast', map="el_rcont")
gs.run_command('d.rast', map="asp_rcont")
gs.run_command('d.vect', map="elev_lid792_cont1m", col="white")
Image(filename="map.png")

### Optional: create TIN model
Convert z-value stored as attribute "value" to z-coordinate.
Compute TIN:

In [ ]:
gs.run_command('v.to.3d', input="elev_lid792_randpts", output="elev_lid792_randpts3d", column="value")
gs.run_command('v.delaunay', input="elev_lid792_randpts3d", output="elev_rand_tin")
gs.mapcalc("level90 = 90")

Visualize the TIN as 3D vector data:
Keep only "level90" and "elev_rand_tin" switched on
(remove or uncheck everything else).
Switch the view from 2D to 3D.
Go to _Data > Vector_ and unckeck _Show vector points_.
In _Vector lines_, change color from black to gray
and set _Display_ from _on surface_ to _as 3D_.




### Optional: Use Python to create the data for IDW comparison

A part of the first interpolation assignment (4A) for GRASS GIS
is computing IDW with different parameters.
The task is to compare the different results in any way you think
is appropriate and comment on the results.
To comment on the results, you need to create them.
You can simply execute the module with different parameters
and then compute all the subsequent analyses which will help you determine
what are the properties of the surfaces generated by IDW with different settings.
However, you can also use Python to compute this data
if you know Python at least a little bit.


The simplest way to execute the Python code
which uses GRASS GIS packages is to use Simple Python editor integrated
in GRASS GIS accessible from the toolbar or the Python tab in the Layer Manager.
Another option is to write the Python code in your favorite plain text editor
like Notepad++ (note that Python editors are plain text editors).
Then run the script in GRASS GIS using the main menu File -> Launch script.


The way to call GRASS modules in Python is very similar to what we use
in the assignments and it is described in the documentation.
The page also contains several examples
how to write simple and more advanced scripts.
Here are some examples relevant to the assignment.


Interpolation using different number of points (modify the list
of numbers of points):

In [ ]:
# execute manually the following or its equivalent:
# import grass.script as gs
# execute manually the following or its equivalent:
# for npoints in [1, 20]:
# execute manually the following or its equivalent:
#     name = 'elev_idw_1m_npoints_{}'.format(npoints)
# execute manually the following or its equivalent:
#     stats = gs.parse_command('v.surf.idw', input='elev_lid792_randpts',
gs.run_command('output=name,', column="value,", npoints="npoints)")

Computing statistics but showing only some for different number of points
(you can combine the code with the code above):

In [ ]:
# execute manually the following or its equivalent:
# import grass.script as gs
# execute manually the following or its equivalent:
# for npoints in [1, 20]:
# execute manually the following or its equivalent:
#     name = 'elev_idw_1m_npoints_{}'.format(npoints)
# execute manually the following or its equivalent:
#     print "\n\n"
# execute manually the following or its equivalent:
#     print name
# execute manually the following or its equivalent:
#     print len(name) * "="
# execute manually the following or its equivalent:
#     stats = gs.parse_command('r.univar', map=name, flags='eg')
# execute manually the following or its equivalent:
#     print stats['min'], stats['max']

Setting the color table and computing shaded relief for changing power
(you need to create the maps before that):

In [ ]:
# execute manually the following or its equivalent:
# import grass.script as gs
# execute manually the following or its equivalent:
# for power in [0.5, 1, 2, 5]:
gs.run_command('gs.run_command(r.colors,')
gs.run_command('map=elev_idw_1m_power_{}.format(power),')
gs.run_command('color=elevation)')
gs.run_command('gs.run_command(r.relief,')
gs.run_command('input=elev_idw_1m_power_{}.format(power),')
gs.run_command('output=elev_idw_1m_power_{}_relief.format(power))')
gs.run_command('gs.run_command(r.shade,')
gs.run_command('color=elev_idw_1m_power_{}.format(power),')
gs.run_command('shade=elev_idw_1m_power_{}_relief.format(power),')
gs.run_command('output=elev_idw_1m_power_{}_shaded.format(power))')

Creating a PNG image with histogram for changing power:

In [ ]:
# execute manually the following or its equivalent:
# import grass.script as gs
# execute manually the following or its equivalent:
# for power in [0.5, 1, 2, 5]:
gs.run_command('gs.run_command(d.mon,', start="cairo,")
gs.run_command('output=elev_idw_1m_power_{}_histogram.png.format(power))')
gs.run_command('gs.run_command(d.histogram,')
gs.run_command('map=elev_idw_1m_power_{}.format(power))')
gs.run_command('gs.run_command(d.mon,', stop="cairo)")

Here are two commands often used when using the scripts.
First is setting the computational. We can do that in a script,
but it better and more general to do it before executing the script:

In [ ]:
gs.run_command('g.region', region="rural_1m")

When we want to run the script again, we first need to remove the created
raster maps:

In [ ]:
gs.run_command('g.remove', type="raster", pattern="elev_idw_1m_npoints_*")

In case you don't know anything about Python scripting
but you still want to try something this might be a good start together
with some (free) courses at
[Codecademy](https://www.codecademy.com/learn/python).
To learn more about using Python in GRASS GIS, see the introduction to the
[grass.script](https://grass.osgeo.org/grass76/manuals/libpython/script_intro.html)
package.

In [ ]:
# end the GRASS session
os.remove(rcfile)